<a href="https://colab.research.google.com/github/lsprietog/public_release/blob/main/demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 IVIM-DKI Machine Learning Demo

This notebook demonstrates how to train and evaluate Machine Learning models for Diffusion MRI analysis, as described in the paper **"Exploring the Potential of Machine Learning Algorithms to Improve Diffusion Nuclear Magnetic Resonance Imaging Models Analysis"**.

**What you will learn:**
1. How to generate synthetic IVIM-DKI signal data.
2. How to train Random Forest and Extra Trees models.
3. How to compare ML performance against standard Least Squares fitting.

---

In [ ]:
# @title 1. Setup & Install
!git clone https://github.com/lsprietog/public_release.git
%cd public_release
!pip install -r requirements.txt

import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.optimize import curve_fit
import joblib

# Define the IVIM-DKI model function
def ivim_dki_model(b, D, f, Dstar, K):
    return f * np.exp(-b * Dstar) + (1 - f) * np.exp(-b * D + (1/6) * (b**2) * (D**2) * K)

In [ ]:
# @title 2. Generate Synthetic Data
# Simulation parameters
n_samples = 5000
b_values = np.array([0, 10, 20, 30, 50, 80, 100, 150, 200, 400, 600, 800, 1000, 1500, 2000])
snr = 30

# Random ground truth parameters
np.random.seed(42)
D_true = np.random.uniform(0.0005, 0.003, n_samples)
f_true = np.random.uniform(0.1, 0.4, n_samples)
Dstar_true = np.random.uniform(0.005, 0.1, n_samples)
K_true = np.random.uniform(0, 2.0, n_samples)

Y_true = np.stack([D_true, f_true, Dstar_true, K_true], axis=1)

# Generate signals
signals = []
for i in range(n_samples):
    sig = ivim_dki_model(b_values, D_true[i], f_true[i], Dstar_true[i], K_true[i])
    # Add Rician noise
    noise_r = np.random.normal(0, 1/snr, len(b_values))
    noise_i = np.random.normal(0, 1/snr, len(b_values))
    noisy_sig = np.sqrt((sig + noise_r)**2 + noise_i**2)
    signals.append(noisy_sig)

X = np.array(signals)
print(f"Generated {n_samples} synthetic signals with SNR={snr}")

In [ ]:
# @title 3. Train Machine Learning Model
print("Training Extra Trees Regressor...")
# Using optimized hyperparameters for speed/size balance
model = ExtraTreesRegressor(
    n_estimators=50,
    max_depth=15,
    min_samples_split=5,
    n_jobs=-1,
    random_state=42
)

model.fit(X, Y_true)
print("Training complete!")

# Evaluate
Y_pred = model.predict(X)
r2 = r2_score(Y_true, Y_pred)
print(f"Model R2 Score: {r2:.4f}")

In [ ]:
# @title 4. Visualize Results
param_names = ['D', 'f', 'D*', 'K']
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

for i in range(4):
    axes[i].scatter(Y_true[:, i], Y_pred[:, i], alpha=0.1, s=5)
    axes[i].plot([Y_true[:, i].min(), Y_true[:, i].max()], 
                 [Y_true[:, i].min(), Y_true[:, i].max()], 'r--')
    axes[i].set_xlabel('Ground Truth')
    axes[i].set_ylabel('Prediction')
    axes[i].set_title(f'{param_names[i]} (R2={r2_score(Y_true[:, i], Y_pred[:, i]):.2f})')

plt.show()

In [ ]:
# @title 5. Save Model
joblib.dump(model, 'my_trained_model.joblib')
print("Model saved as 'my_trained_model.joblib'")
files.download('my_trained_model.joblib')